# Average of average neighbor distance

### Set values for whether you would like to generate new results or just use the JSON that has previously been generated

In [1]:
# Set to true if you want to run the tests again. Otherwise just loads results from JSON
GENERATE_NEW_RESULTS = False

# Set to true if you want to save figures to disk. Change path as needed
SAVE_FIGURES_TO_DISK = False
FIG_SAVE_PATH = "../thesis/figures"

Load in packages and set options for prettier plots

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import concurrent.futures
import json
import numpy as np

from test_results_helper_funcs import save_fig, make_cache, get_val_arr, MORTON, ROW_ARR, BLOCK_ARR

from data_structures.morton_order import MortonOrder
from data_structures.block_array import BlockArray
from data_structures.row_major_array import RowMajorArray

## Defining functions/variables to run tests

In [3]:
def avg_dist_to_neighbors_from_voxel(element, x, y, z):
    """
    Calculates the average distance to neighbors in linear address space
    at a specific voxel
    """
    internal_index = element.internal_index(x, y, z)
    summ, elements_counted = 0, 0
    for x_inc in range(-1, 2):
        for y_inc in range(-1, 2):
            for z_inc in range(-1, 2):
                x_here = x + x_inc
                y_here = y + y_inc
                z_here = z + z_inc
                if x_inc == 0 and y_inc == 0 and z_inc == 0 or \
                   not element.valid_index(x_here, y_here, z_here):
                    continue
                index_here = element.internal_index(x_here, y_here, z_here)
                summ += abs(internal_index - index_here)
                elements_counted += 1
    return float(summ) / elements_counted

def avg_of_avg_dist_to_neighbors(element):
    """
    Calculates the average of all average distances in linear
    address space to neighbors in a data structure
    """
    summ = 0.0
    for (x, y, z) in element.iter_keys():
        summ += avg_dist_to_neighbors_from_voxel(element, x, y, z)
    return summ / np.prod(element.shape)

In [4]:
def avg_dist_for_n_and_data(vals):
    name, arr = vals
    avg = avg_of_avg_dist_to_neighbors(arr)
    print(f"{name} {arr.shape} finished")
    return name, avg

def avg_dist_for_n(n):
    rnd_vals = np.random.rand(n, n, n)
    test_vals = [
        (MORTON, MortonOrder(rnd_vals)),
        (BLOCK_ARR, BlockArray(rnd_vals, S=(4 if n <= 8 else 8))),
        (ROW_ARR, RowMajorArray(rnd_vals))
    ]
    res = {}
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for (name, avg) in executor.map(avg_dist_for_n_and_data, test_vals):
            res[name] = avg
    return res, n

In [5]:
ns = [2**i for i in range(2, 9)]
print(ns)

[4, 8, 16, 32, 64, 128, 256]


## Generating/loading in test results

In [6]:
if GENERATE_NEW_RESULTS:
    results = {}
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for (obj, n) in executor.map(avg_dist_for_n, ns):
            results[n] = obj

    with open('results/avg-avg-neighbor-dist.json', 'w') as f:
        json.dump(results, f, indent=4)
else:
    with open('results/avg-avg-neighbor-dist.json', 'r') as f:
        results = json.load(f)

## Print results to copy-pasteable $\LaTeX$-table

In [8]:
# Print the results in such a way that they can be copy-pasted to LaTeX table
for n in results:
    vals = results[n]
    print(f"{n:>3} & {vals[MORTON]:>8.2f} & {vals[BLOCK_ARR]:>8.2f} & {vals[ROW_ARR]:>8.2f} \\\\")

  4 &     9.54 &    11.21 &    11.21 \\
  8 &    39.78 &    40.54 &    44.26 \\
 16 &   166.61 &   169.06 &   176.86 \\
 32 &   685.82 &   690.76 &   707.89 \\
 64 &  2787.28 &  2802.51 &  2833.36 \\
128 & 11243.36 & 11282.46 & 11337.83 \\
256 & 45169.30 & 45258.10 & 45360.92 \\
